# Exercise: 1D Burgers Equation

The Cauchy Problem to be solved is the Burgers Equation 
\begin{align}
&\partial_t u (x,t) + u \partial_x u (x,t) - \nu \partial_x^2 u (x,t) = 0 \\
&u(t=0, x) = - \sin \left( \pi \, x \right) \\
&u\left(t, x=\pm 1\right) =  0
\end{align}
with $(t,x) \in [0,1] \times [-1, +1]$, and where we set
$$
\nu = \frac{0.01}{\pi}
$$


------
Extra: reviews on PINN

[1] https://www.nature.com/articles/s42254-021-00314-5

[2] https://arxiv.org/pdf/2202.06416.pdf

[3] https://www.mdpi.com/2504-2289/6/4/140

[4] https://medium.com/@vignesh.g1609/pinn-physics-informed-neural-networks-5f5f05bf7231

[5] https://ocw.mit.edu/courses/18-152-introduction-to-partial-differential-equations-fall-2011/29c6f7ee914a1d804899781f9f604f49_MIT18_152F11_lec_24.pdf , https://ocw.mit.edu/courses/18-152-introduction-to-partial-differential-equations-fall-2011/download/

In [ ]:
# Find GPU in this environment
import os, subprocess, re
os.environ["CUDA_VISIBLE_DEVICES"] = ''.join(re.findall("UUID: (MIG-[^)]+)\)", str(subprocess.check_output(["nvidia-smi", "-L"]), 'ascii')))

In [ ]:
from typing import Type, Union
import gc
import tqdm
import math
from collections import OrderedDict
try:
    import tqdm
except:
    %pip install tqdm
    import tqdm

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
import matplotlib.pyplot as plt

## 1. Define a DNN as torch module

In [ ]:
class PINN_DNN(nn.Module):
    """
    Basic Deep Neural Network
    """
    def __init__(
        self,
        # 
    ):
        super(PINN_DNN, self).__init__()
        #---

    def forward(self, x):
        return # ---

## 2. Define the PDE class with torch.autograd

In [ ]:
class Burgers_1D_PDE:
    """
        Class computing the 1D Burgers PDE equation:
            u_t + u u_x - ν u_xx = 0
        Notice that we can conveniently rewrite it as
            u_t + (1/2)*(u**2)_x- ν u_xx = 0

        --------------------------
        Args:
            nu    (float): Float parameter describing the diffusion term

        Methods:
            compute_heat      (coords, pred_funcs): returns the computed Heat eq.
    """
    def __init__(
        self,
        nu: float = float(0.01/np.pi),     # PDE coefficient
    ):
        self.nu = nu

    def compute_burgers(self, coords: torch.Tensor, pred_func: torch.Tensor) -> torch.Tensor:
        """
        Compute 1D Heat equation
            u_t + (u**2)_x/2 - ν u_xx
        It should equate to zero.

        Args:
            coords    (torch.Tensor): (t,x) coords.
            pred_func (torch.Tensor): (u) evaluated at (t,x)
        """
        # ----

    def get_derivative(self, y, x, n: int = 1):
        """
        General formula to compute the n-th order derivative of y = f(x) with respect to x
        """
        if n == 0:
            return y
        else:
            dy_dx = torch.autograd.grad(y, x, torch.ones_like(y).to(y.device), create_graph=True, retain_graph=True, allow_unused=True)[0]
        return self.get_derivative(dy_dx, x, n - 1)

### 2.0 test the PDE class

In [ ]:
"""
    Test the Burgers_1D_PDE class
"""
nu_param = float(0.01/np.pi)

# .....

print(f"Pred: {heat_pred.mean()}")

## 3. Boundary Conditions

In [ ]:
class Burgers_1D_BC:
    def __init__(
        self,
        cost_function = nn.MSELoss()
    ):
        # Cost function
        self.cost_function = cost_function

    def boundary_cond(self, coords: torch.Tensor, pred_func: torch.Tensor) -> torch.Tensor:
        """
        Class for computing BC
            u(t, x=±1) = 0

        Args:
             coords    (torch.Tensor) : The coords at the spatial boundary
             pred_func (torch.Tensor) : The predicted function at those points
        Returns
             bc_loss (torch.tensor)
        """
        #....

    def initial_cond(self, coords: torch.Tensor, pred_func: torch.Tensor) -> torch.Tensor:
        """
        Class for computing IC
            u(x, t=0) = - sin(π x)

        Args:
             coords    (torch.Tensor) : The coords at the Initial Time
             pred_func (torch.Tensor) : The predicted function at those points
        Returns
             ic_loss (torch.tensor)
        """
        #....

### 3.0 Test Boundary Class

In [ ]:
"""
    Test the BC_PDE class
"""
# ....

In [ ]:
X = Variable(ic_coords, requires_grad=True)

test_pde.get_derivative(- torch.sin(np.pi * X), X, 1)[:,1] + np.pi* torch.cos(np.pi * X[:,1])

test_pde.get_derivative( 
    test_bc.initial_cond(
        X, - torch.sin(np.pi * X[:,1]).unsqueeze(-1)
    ), X
)

## 4. Assemble PINN model

--------------
Refs:

[1] https://towardsdatascience.com/improving-pinns-through-adaptive-loss-balancing-55662759e701 , https://github.com/rbischof/relative_balancing , https://arxiv.org/abs/2110.09813

[2] https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/user_guide/theory/advanced_schemes.html#softadapt , original paper: https://arxiv.org/pdf/1912.12355.pdf

[3] https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/user_guide/theory/advanced_schemes.html#learning-rate-annealing . original paper https://arxiv.org/pdf/2001.04536.pdf



In [ ]:
import json

def write_line_to_file(LOG_FILE: str, log_line: str):
    with open(LOG_FILE, 'a') as f:
        f.write(log_line)

def store_hyp_dict(json_file: str, hyperparam_kwargs: dict, _indent: int = 4):
    with open(json_file, 'w') as fp:
        json.dump(hyperparam_kwargs, fp, indent=_indent)

In [ ]:
# Checkpoints (to save model parameters during training)
# this is implemented by writing a python class that uses the torch.save method
class SaveBestModel:
    def __init__(
        self,
        model_name: str = 'best_model',
        best_valid_loss=float('inf')
    ): #object initialized with best_loss = +infinite
        self.best_valid_loss = best_valid_loss
        self.model_name = model_name

    def __call__(
        self, current_valid_loss,
        epoch, model, optimizer, criterion
    ):
        if current_valid_loss < self.best_valid_loss:
            self.best_valid_loss = current_valid_loss
            print(f"\nBest validation loss: {self.best_valid_loss}")
            print(f"\nSaving best model for epoch: {epoch}\n")
            # method to save a model (the state_dict: a python dictionary object that
            # maps each layer to its parameter tensor) and other useful parametrers
            # see: https://pytorch.org/tutorials/beginner/saving_loading_models.html
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, f'{self.model_name}.pth')

In [ ]:
import datetime
import time

class PINN(nn.Module):
    """
    Full PINN Class for the FORWARD problem.

    It incorporates:
        1. The PINN DNN (in self.DNN)
        2. The PDE      (in self._PDE)
        3. The BC       (in self._BC)

    It exposes the train_model() method to solve the PINN forward problem, by Performing a training with ADAM optimiser

    The train_model() method is thus:
        1. ADAM loop
            1.1. call training_step() for adam
            1.2. Store best model
            1.3. Perform step for learning rate stepper
            1.4. Logs
            1.5. Check if patience reached;

    Args:
        For Args see init method.

    Methods:
        init_model      ()  : init the model.
        load_best_model ()  : reload the best model.
        forward         (x) : DNN Forward passing
        pde_loss        (x, pred_funcs) : Compute PDE loss using self.cost_function
        ic_loss         (x, pred_funcs) : Compute IC  loss using BC class
        bc_loss         (x, pred_funcs) : Compute BC  loss using BC class
        soft_adapt      (losses, previous_losses, eps = 1e-8) : perform SoftAdapt algorithm.
        generate_coords ()  : generate the coords
        closure         ()  : method performing the backprop. Fundamental for L-BFGS part, it is also used for ADAM.
        training_step   (epoch, use_adam)   : Single training step.
        train_model     ()  : Main method. Trains the model.
        store_training_df   ()  : utils method to store the training history as csv
    """
    def __init__(
        self,
        use_rec: bool = True, # parameter to flag False if PINN solve PDE without reconstruction info
        # DNN
        n_inputs : int = 2,      # number of inputs, e.g. x,y,z,t,....
        n_outputs: int = 1,      # number of outputs, e.g. u,v,P, T, ....
        hidden_layers: list = [4, 8, 16, 8], # number of hidden layers
        dropout: float = 0.2,  # Dropout
        activation_func = nn.Tanh(),
        learning_rate: float = 0.001,
        # True func
        exact_solution_func = lambda x: x,
        # Geometry
        time_interval : list = [0.0,  +1.0] ,
        space_interval: list = [-1.0, +1.0] ,
        # PDE
        diffusion_coefficient: float = float(0.01/np.pi),     # PDE coefficient
        # Training
        patience_training: int = 200,
        epochs: int = 1000,
        N_batches: int = 32,
        patience_lr: int = 50,
        # Soft adapt
        use_softadapt : bool = False,
        softadapt_starting_epoch : int = 5,
        # Loss weights
        weight_rec: float = 1.0,
        weight_pde: float = 1.0,
        weight_bc : float = 1.0,
        weight_ic : float = 1.0,
        # dataloaders
        fun_batch_size: int = 4096,
        pde_batch_size: int = 4096,
        bc_batch_size: int = 1024,
        ic_batch_size: int = 1024,
        # device
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
        # Model name
        BASE_PATH_TO_STORE: str = '.',
        model_name: str = 'pinne_heat',
    ):
        """
        Init method.

        Args:
            use_rec     (bool, optional)    : Boolean to pick if rec_loss should be used. Defaults to True.
            n_inputs    (int, optional)     : Number of DNN inputs. Defaults to 2.
            n_outputs   (int, optional)     : Number of DNN outputs. Defaults to 1.
            hidden_layers (list, optional)  : Listof hidden layers dims. Defaults to [4, 8, 16, 8].
            learning_rate (float, optional) : Adam Learning rate. Defaults to 0.001.
            exact_solution_func     (function, optional): Method to generate the exact solution. Defaults to exact_solution_func.
            diffusion_coefficient   (float, optional)   : Heat diffusion coefficient. Defaults to 0.01/π.
            patience_training       (int, optional)     : Patience in ADAM training. Defaults to 200.
            epochs                  (int, optional)     : Total Number of ADAM epochs. Defaults to 1000.
            N_batches               (int, optional)     : Number of ADAM Batches. Defaults to 32.
            patience_lr             (int, optional)     : Patience for LR stepper. Defaults to 50.
            use_softadapt            (bool, optional) : Boolean to decide whether to use SoftAdapt algorithm. Defaults to True,
            softadapt_starting_epoch (int, optional)  : SoftAdapt starting epoch. Defaults to 5.
            weight_rec  (float, optional)   : Reconstruction Loss weight. Defaults to 1.0.
            weight_pde  (float, optional)   : PDE Loss weight. Defaults to 1.0.
            weight_bc   (float, optional)   : BC Loss weight. Defaults to 1.0.
            weight_ic   (float, optional)   : IC Loss weight. Defaults to 1.0.
            fun_batch_size  (int, optional) : Batch Size for computing rec loss. Defaults to 4096.
            pde_batch_size  (int, optional) : Batch Size for computing pde loss. Defaults to 4096.
            bc_batch_size   (int, optional) : Batch Size for computing bc  loss. Defaults to 1024.
            device      (_type_, optional)  : Device. Defaults to torch.device("cuda:0" if torch.cuda.is_available() else "cpu").
            BASE_PATH_TO_STORE  (str, optional) : Path to store training and model data. Defaults to '.'.
            model_name          (str, optional) : Model name. Defaults to 'pinne_heat'.
        """
        super(PINN, self).__init__()
        self.use_rec = use_rec
        # geometry
        self._t_min , self._t_max = time_interval
        self._x_min , self._x_max = space_interval
        # ==== DNN PART ==================
        self.n_inputs      = n_inputs
        self.n_outputs     = n_outputs
        self.hidden_layers = hidden_layers
        self.n_layers      = len(hidden_layers)
        self.dropout_prob  = dropout
        self.activation_func = activation_func
        self.learning_rate = learning_rate
        self.device = device
        # ==== SoftAdapt PART ==================
        self.use_softadapt = use_softadapt
        self.softadapt_starting_epoch = softadapt_starting_epoch if softadapt_starting_epoch >= 2 else 2
        # ====  PDE params PART ==================
        self.diffusion_coefficient = diffusion_coefficient
        self.exact_solution_func   = exact_solution_func

        # ==== PINN-DNN PART ==================
        # Dataloader
        self.fun_batch_size = fun_batch_size
        self.pde_batch_size = pde_batch_size
        self.ic_batch_size  = ic_batch_size
        self.bc_batch_size  = bc_batch_size
        self.N_batches   = N_batches
        # training vars
        self.patience_training = patience_training
        self.epochs = epochs
        self.patience_lr     = patience_lr
        # Loss weights
        self.weight_rec = weight_rec
        self.weight_pde = weight_pde
        self.weight_bc  = weight_bc
        self.weight_ic  = weight_ic
        # DNN
        self.model_kwargs = {
           "n_inputs"     : self.n_inputs,
            "n_outputs"   : self.n_outputs,
            "hidden_dims" : self.hidden_layers,
            "dropout"     : self.dropout_prob,
        }
        self.init_model()
        # Optimisers
        self.optimizer = torch.optim.Adam(
            self.DNN.parameters(),
            lr=self.learning_rate,
        )
        # LR scheduler
        self.lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            factor  = 0.1, # Factor by which the learning rate will be reduced. new_lr = lr * factor. Default: 0.1.
            patience= self.patience_lr, #  Number of epochs with no improvement after which learning rate will be reduced.
        )

        # cost function
        self.cost_function = nn.MSELoss() # Mean squared error

        # ==== PDE PART ==================
        # PDE+BC
        self._PDE = Burgers_1D_PDE(nu=self.diffusion_coefficient)
        self._BC  = Burgers_1D_BC(
            cost_function = self.cost_function
        )

        # save best model
        self.BASE_PATH_TO_STORE = BASE_PATH_TO_STORE
        self.model_name = model_name
        self.full_path_to_store = f"{BASE_PATH_TO_STORE}/{model_name}"
        self.save_best_model = SaveBestModel(model_name=self.full_path_to_store) #initialize checkpoint function

        # Storing
        # === STORE HYPERPARAMETERS =====
        self.hyperparam_kwargs = {
            # Model kwargs
            **self.model_kwargs,
            "activation_func" : f"{self.activation_func}",
            # Dataset info
            'train_size': (self.pde_batch_size+self.bc_batch_size)*self.epochs,
            # Hyperparameters
            'epochs'   : self.epochs,
            'patience' : self.patience_training,
            'lr_patience': self.patience_lr
        }
        store_hyp_dict(f'{self.full_path_to_store}.json', self.hyperparam_kwargs)
        # logs
        self.LOG_FILE = f"{self.full_path_to_store}.txt"

        # === Training vars =====
        self.training_rec_loss = []
        self.training_pde_loss = []
        self.training_bc_loss  = []
        self.training_ic_loss  = []

        self.training_loss   = []
        self.learning_rates  = []

        self.weight_rec_history = []
        self.weight_pde_history = []
        self.weight_bc_history  = []
        self.weight_ic_history  = []

        self.Delta_training = 0 # Delta value for L-FBGS traingin

        self.current_epoch = 0 # store current epoch to pass to closure for soft-adapt

    def init_model(self):
        self.DNN = PINN_DNN(
            activation_func=self.activation_func,
            **self.model_kwargs,
        ).to(self.device)

    def load_best_model(self):
        try:
            # load the best model
            RELOAD_MODEL_NAME = f"{self.full_path_to_store}.pth"
            checkpoint = torch.load(RELOAD_MODEL_NAME)
            self.init_model()
            self.DNN.load_state_dict(checkpoint['model_state_dict'])
            print(f"Loaded best model: {RELOAD_MODEL_NAME} at epoch: {checkpoint['epoch']}\n")
        except Exception as e:
            print(f"Impossible to load {RELOAD_MODEL_NAME}\nError: {e}\n")
            pass

    def forward(self, x):
        """
        Forward step via DNN
        """
        pred_funcs = self.DNN(x)
        return pred_funcs

    def pde_loss(self, x, pred_funcs):
        """
        Method to compute the PDE loss;
        the PDE class gives the evaluation of the LHS of the PDE system, i.e.
            PDE[u](t,x) = 0
        Here we use the cost function to compute the loss.
        """
        burgers_eq = self._PDE.compute_burgers(x, pred_funcs)

        # The PDE loss is the mean of the squared (0 - PDE)^2 -- see nVidia Modulus example docs
        loss_pde  = self.cost_function(
            burgers_eq,
            torch.zeros_like(burgers_eq).to(burgers_eq.device)
        )

        return loss_pde

    def bc_loss(self, x, preds_funcs):
        """
        Method to compute the Boundary Condition Loss.
        It is already implemented into the BC class, so we simply invoke it.
        """
        bc_loss = self._BC.boundary_cond(x, preds_funcs)
        return bc_loss

    def ic_loss(self, x, preds_funcs):
        """
        Method to compute the Initial Condition Loss.
        It is already implemented into the BC class, so we simply invoke it.
        """
        ic_loss = self._BC.initial_cond(x, preds_funcs)
        return ic_loss

    def soft_adapt(self, losses: list, previous_losses: list, eps: float = 1e-8) -> list:
        """
        Performs the softadapt computation:

        Args:
            losses          (list) : list of losses to compute softadapt on
            previous_losses (list) : list of losses at previous step
            eps (float) : Factor to avoid division by zero. Defaults to 1e-8

        Returns
            (list) list of W params
        """
        Li = np.array(losses)
        Lo = np.array(previous_losses)

        _ratio = (Li)/(Lo + eps)
        _mu = np.max(_ratio)

        _ratio = torch.tensor(_ratio, requires_grad=False)

        _w = nn.functional.softmax(_ratio - _mu)

        return _w.tolist()

    def generate_coords(self):
        """
        Method to generate the coords.
        Notice that we have implemented here a RANDOM EXTRACTOR.

        Returns
            coords    (torch.Tensor) : bulk coordinates
            ic_coords (torch.Tensor) : IC coordinates
            bc_coords (torch.Tensor) : BC coordinates
        """
        coords = torch.cat(
            (
                self._t_min + (self._t_max - self._t_min)*torch.rand(self.pde_batch_size).unsqueeze(-1), # t
                self._x_min + (self._x_max - self._x_min)*torch.rand(self.pde_batch_size).unsqueeze(-1)  # x
            ),
            dim=-1
        )
        coords = Variable(coords.float(), requires_grad=True)
        # initial cond
        ic_coords = torch.cat(
            (
                self._t_min * torch.ones(self.ic_batch_size).unsqueeze(-1), # t
                self._x_min + (self._x_max - self._x_min)*torch.rand(self.ic_batch_size).unsqueeze(-1)  # x
            ),
            dim=-1
        )
        # bc
        bc_coords_p = torch.cat(
            (
                self._t_min + (self._t_max - self._t_min)*torch.rand(self.bc_batch_size//2).unsqueeze(-1), # t
                self._x_max*torch.ones(self.bc_batch_size//2).unsqueeze(-1)  # x
            ),
            dim=-1
        )
        bc_coords_m = torch.cat(
            (
                self._t_min + (self._t_max - self._t_min)*torch.rand(self.bc_batch_size//2).unsqueeze(-1), # t
                self._x_min*torch.ones(self.bc_batch_size//2).unsqueeze(-1)  # x
            ),
            dim=-1
        )
        bc_coords = torch.cat([bc_coords_m, bc_coords_p])

        ic_coords = Variable(ic_coords.float(), requires_grad=True)
        bc_coords = Variable(bc_coords.float(), requires_grad=True)
        # to device
        coords     = coords.to(self.device)
        ic_coords  = ic_coords.to(self.device)
        bc_coords  = bc_coords.to(self.device)

        return coords, ic_coords, bc_coords

    def closure(self):
        """
        Example of closure func:

        if torch.is_grad_enabled():
            self.lbfgs_optimizer.zero_grad()
        output = self(X_)
        loss = self.lossFct(output, y_)
        if loss.requires_grad:
            loss.backward()
        return loss
        """
        self.optimizer.zero_grad()
        # =================== random extraction =====================
        # func + pde
        coords, ic_coords, bc_coords = self.generate_coords()
        # =================== forward =====================
        true_funcs = self.exact_solution_func(coords)# true
        pred_funcs = self.forward(coords)            # DNN pred
        pred_ic    = self.forward(ic_coords)         # DNN pred - ic
        pred_bc    = self.forward(bc_coords)         # DNN pred - bc
        # =================== Losses =====================
        pde_loss = self.pde_loss(coords, pred_funcs)                                                                        # PDE loss
        ic_loss  = self.ic_loss(ic_coords, pred_ic)                                                                         # BC loss
        bc_loss  = self.bc_loss(bc_coords, pred_bc)                                                                         # BC loss
        rec_loss = self.cost_function(pred_funcs, true_funcs.unsqueeze(-1)) if self.use_rec else torch.zeros_like(pde_loss) # rec loss
        # =================== soft-adapt =================
        if self.use_softadapt and self.current_epoch >= self.softadapt_starting_epoch:
            if self.use_rec:
                _losses   =  [rec_loss.item(), pde_loss.item(), ic_loss.item(), bc_loss.item() ]
                _p_losses =  [self.training_rec_loss[-1], self.training_pde_loss[-1], self.training_ic_loss[-1], self.training_bc_loss[-1] ]
                self.weight_rec, self.weight_pde, self.weight_ic, self.weight_bc = self.soft_adapt(
                    losses          = _losses ,
                    previous_losses = _p_losses
                )
            else:
                _losses   = [pde_loss.item(), ic_loss.item(), bc_loss.item() ]
                _p_losses = [self.training_pde_loss[-1],  self.training_ic_loss[-1], self.training_bc_loss[-1] ]
                self.weight_pde, self.weight_ic, self.weight_bc = self.soft_adapt(
                    losses          = _losses ,
                    previous_losses = _p_losses
                )

        # full loss
        loss = self.weight_pde * pde_loss + self.weight_ic * ic_loss  + self.weight_bc * bc_loss  # <=== Full loss here ====
        if self.use_rec:
            loss += self.weight_rec * rec_loss
        # Normalise loss
        loss = loss/( self.weight_rec + self.weight_pde + self.weight_ic + self.weight_bc ) if  self.use_rec else  loss/( self.weight_pde + self.weight_ic + self.weight_bc )

        # Append
        self.train_loss += loss.item()
        self.train_rec_loss += rec_loss.item()
        self.train_pde_loss += pde_loss.item()
        self.train_ic_loss  += ic_loss.item()
        self.train_bc_loss  += bc_loss.item()
        #==== backward ======================
        loss.backward(retain_graph=True)

        return loss

    def training_step(self, epoch: int, use_adam: bool = True):
        """
        Single Epoch training step
        """
        # =====================================================
        # Training
        self.DNN.train()
        self.train_loss = 0
        self.train_rec_loss = 0
        self.train_pde_loss = 0
        self.train_ic_loss  = 0
        self.train_bc_loss  = 0
        # =================== Batches iterations - only ADAM =====================
        for _ in tqdm.tqdm(range(self.N_batches)):
            # =================== closure ====================
            self.optimizer.step(self.closure)
        # =================== compute LOSS ====================
        # set divisor for batches
        _divisor = self.N_batches
        # training loss
        tr_loss = self.train_loss/_divisor
        # others
        tr_rec_loss = self.train_rec_loss/_divisor
        tr_pde_loss = self.train_pde_loss/_divisor
        tr_ic_loss  = self.train_ic_loss /_divisor
        tr_bc_loss  = self.train_bc_loss /_divisor
        # === Append Losses ========
        self.training_loss.append(tr_loss) ## Full trainloss
        # others
        self.training_rec_loss.append(tr_rec_loss)
        self.training_pde_loss.append(tr_pde_loss)
        self.training_ic_loss.append(tr_ic_loss)
        self.training_bc_loss.append(tr_bc_loss)
        # append weights
        self.weight_rec_history.append(self.weight_rec)
        self.weight_pde_history.append(self.weight_pde)
        self.weight_ic_history.append(self.weight_ic)
        self.weight_bc_history.append(self.weight_bc)

        return tr_loss

    def train_model(self):
        t0 = time.time()

        self.training_rec_loss = []
        self.training_pde_loss = []
        self.training_ic_loss  = []
        self.training_bc_loss  = []

        self.training_loss   = []
        self.learning_rates  = []
        #==== ADAM TRAINING LOOP ================================================================================
        for epoch in range(0, self.epochs):
            self.current_epoch = epoch
            loss = self.training_step(epoch)

            #=== GO ON ====
            #save best model
            self.save_best_model(loss, epoch, self.DNN, self.optimizer, self.cost_function)

            # Learning Rate stepper
            current_lr = self.optimizer.param_groups[0]['lr']
            self.learning_rates.append(current_lr)
            # update learning rate schedule
            self.lr_scheduler.step(loss) ### NB: ONLY FOR ReduceLROnPlateau

            # =================== log ========================
            log_line = f'====> Epoch: {epoch}\tTraining loss: {loss:.6f}\tlr: {current_lr:.2e}\tTime: {datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")}\n'
            print(log_line)
            write_line_to_file(LOG_FILE=self.LOG_FILE, log_line=log_line)

            # update number of epochs passed
            self.Delta_training += 1
            # Check patience
            if self.patience_training > 0 and len(self.training_loss) - np.array(self.training_loss).argmin() > self.patience_training:
                break_log = f"\nPatience treshold = {self.patience_training} reached.\nExiting at epoch {epoch}.\n"
                print(break_log)
                write_line_to_file(LOG_FILE=self.LOG_FILE, log_line=break_log)
                break

        # Close up
        log_line = f'\n\nTotal ADAM training time: {time.time() - t0}\tEnd time: {datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")}\n\n'
        print(log_line)
        write_line_to_file(LOG_FILE=self.LOG_FILE, log_line=log_line)

        # store history df
        self.store_training_df()

    def store_training_df(self):
        # store as pandas csv
        df_train = pd.DataFrame(
            {
                "epochs"          : [ epoch for epoch in range(len(self.training_loss)) ],
                "training_loss"   : self.training_loss,
                'lr'              : self.learning_rates,
                "rec_train_losses": self.training_rec_loss,
                "pde_train_losses": self.training_pde_loss,
                "ic_train_losses" : self.training_ic_loss,
                "bc_train_losses" : self.training_bc_loss,
                "weight_rec"      : self.weight_rec_history,
                "weight_pde"      : self.weight_pde_history,
                "weight_ic"       : self.weight_ic_history,
                "weight_bc"       : self.weight_bc_history,
            }
        )
        df_train.to_csv(f'{self.BASE_PATH_TO_STORE}/{self.model_name}_history.csv')

### 4.0 Try model implementation

In [ ]:
try:
    from torchsummary import summary
except:
    %pip install torchsummary
    from torchsummary import summary

nu_param = float(0.01/np.pi)

pinn_model = PINN(
    time_interval = [0.0,  +1.0] ,
    space_interval= [-1.0, +1.0] ,
    diffusion_coefficient =nu_param,
    hidden_layers = [
        #128, 128, 128
        #40,40,40,40,40,40
        64, 64, 64, 64, 64
    ],
    activation_func=nn.GELU(), #nn.Tanh(), 
    exact_solution_func=lambda x: x ,
    learning_rate = 0.001,
    BASE_PATH_TO_STORE='./model_data',
    model_name = 'pinne_burgers_prova',
    use_rec = False,
    #use_softadapt = False,
    weight_rec = 1.0,
    weight_pde = 1.0,
    weight_bc  = 1.0,
    weight_ic  = 1.0,
    #weight_pde = 0.5,
    #weight_bc  = 1.5,
    #weight_ic  = 4.5,
    pde_batch_size = 4096,
    bc_batch_size  = 4096,
    ic_batch_size  = 4096,
    # Training
    patience_training = 200,
    epochs      = 1000,
    N_batches   = 32,
    patience_lr = 50,
    )

print(pinn_model.DNN)

print(f"\n\nTorchSummary:\n")
summary(pinn_model.DNN, input_size=( 2, ), batch_size=4096, device=pinn_model.device.type)

## 5. Train model

In [ ]:
pinn_model.train_model()

### 5.1. Training History

## 6. Test model

In [ ]:
# load the best model
pinn_model.load_best_model()